In [43]:
from networkx.algorithms import bipartite
import networkx as nx
from networkx import *
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [44]:
# 80000row
user_train = pd.read_csv('ml-100k/u5.base', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_train = user_train[['userID','itemID','rating']]

# 100000row
user_total = pd.read_csv('ml-100k/u.data', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_total = user_total[['userID','itemID','rating']]

# 20000row
user_test = pd.read_csv('ml-100k/u5.test', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_test = user_test[['userID','itemID','rating']]


In [45]:
'''
min_max_scaler = MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(user_train[['rating']])
user_train[['rating']] = x_scaled

x_scaled = min_max_scaler.fit_transform(user_total[['rating']])
user_total[['rating']] = x_scaled

x_scaled = min_max_scaler.fit_transform(user_test[['rating']])
user_test [['rating']] = x_scaled
'''

"\nmin_max_scaler = MinMaxScaler()\n\nx_scaled = min_max_scaler.fit_transform(user_train[['rating']])\nuser_train[['rating']] = x_scaled\n\nx_scaled = min_max_scaler.fit_transform(user_total[['rating']])\nuser_total[['rating']] = x_scaled\n\nx_scaled = min_max_scaler.fit_transform(user_test[['rating']])\nuser_test [['rating']] = x_scaled\n"

In [46]:
user_item_total = user_total.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)
user_item_train = user_train.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0) # train 용
user_item_test = user_test.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)

total_user_idx = user_item_total.index

In [47]:
## user-item node 이름 설정

matrix = pd.read_csv('ml-100k/u.data', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
matrix[['userID']] = 'u' + matrix[['userID']].astype(str)
matrix[['itemID']] = 'i' + matrix[['itemID']].astype(str)
matrix

,userID,itemID,rating,timestamp
0,u196,i242,3,881250949
1,u186,i302,3,891717742
2,u22,i377,1,878887116
3,u244,i51,2,880606923
4,u166,i346,1,886397596
...,...,...,...,...
99995,u880,i476,3,880175444
99996,u716,i204,5,879795543
99997,u276,i1090,1,874795795
99998,u13,i225,2,882399156


In [48]:
## user-item node 이름 설정

matrix = pd.read_csv('ml-100k/u.data', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
matrix[['userID']] = 'u' + matrix[['userID']].astype(str)
matrix[['itemID']] = 'i' + matrix[['itemID']].astype(str)
matrix

,userID,itemID,rating,timestamp
0,u196,i242,3,881250949
1,u186,i302,3,891717742
2,u22,i377,1,878887116
3,u244,i51,2,880606923
4,u166,i346,1,886397596
...,...,...,...,...
99995,u880,i476,3,880175444
99996,u716,i204,5,879795543
99997,u276,i1090,1,874795795
99998,u13,i225,2,882399156


In [49]:
import numpy as np
user_node = matrix[['userID']].values
user_node = np.array(user_node).flatten().tolist()
#user_node

In [50]:
G = nx.Graph()
G.add_nodes_from(user_node)
info(G)

C:\Users\kddin\AppData\Local\Temp\ipykernel_1992\1259828340.py:3: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  info(G)


'Graph with 943 nodes and 0 edges'

In [51]:
edgelist = []

for i in matrix.values:
    edgelist.append((i[0],i[1]))
    
B = nx.Graph() # >300
B.add_nodes_from(matrix.userID, bipartite=0) # user
B.add_nodes_from(matrix.itemID, bipartite=1) # item(movie)
B.add_edges_from(edgelist)

info(B)

C:\Users\kddin\AppData\Local\Temp\ipykernel_1992\700776426.py:11: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  info(B)


'Graph with 2625 nodes and 100000 edges'

## Link Prediction

 전체 그래프에서 상위 1%의 연결 가능성을 해당하는 link connection

In [52]:
for i in range(10):
    
    user_edgelist = []
    total_edgelist = []
    
    pred_link = list(nx.common_neighbor_centrality(B, alpha=0.8))
    
    link_likelihood = pd.DataFrame(pred_link).iloc[:,2].values
    link_99 = np.percentile(link_likelihood,  99.5, interpolation='linear')
    
    for p in pred_link:
        edge = list(p)
        linklihood = int(edge[2])
        
        if linklihood > link_99:
            if edge[0][:1] ==  edge[1][:1] == 'u': # user R
                user_edgelist.append((edge[0], edge[1]))
                total_edgelist.append((edge[0], edge[1]))
            else :
                total_edgelist.append((edge[0], edge[1]))
                
    B.add_edges_from(total_edgelist)
    G.add_edges_from(user_edgelist)
    
    print('total_graph connected : ', is_connected(B),' user graph info : ', info(B))
    print('user_graph connected : ', is_connected(G),' user graph info : ', info(G))
    
    if is_connected(G):
        break
    
#print('total graph: ', info(B))
#print('user_graph connected : ', is_connected(G),' user graph info : ', info(G))

C:\Users\kddin\AppData\Local\Temp\ipykernel_1992\2736348787.py:9: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they. (Deprecated NumPy 1.22)
  link_99 = np.percentile(link_likelihood,  99.5, interpolation='linear')
C:\Users\kddin\AppData\Local\Temp\ipykernel_1992\2736348787.py:25: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print('total_graph connected : ', is_connected(B),' user graph info : ', info(B))
C:\Users\kddin\AppData\Local\Temp\ipykernel_1992\2736348787.py:26: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print('user_graph connected : ', is_connected(G),' user graph info : ', info(G))


total_graph connected :  True  user graph info :  Graph with 2625 nodes and 116425 edges
user_graph connected :  False  user graph info :  Graph with 943 nodes and 8702 edges
total_graph connected :  True  user graph info :  Graph with 2625 nodes and 132859 edges
user_graph connected :  False  user graph info :  Graph with 943 nodes and 15777 edges
total_graph connected :  True  user graph info :  Graph with 2625 nodes and 149296 edges
user_graph connected :  False  user graph info :  Graph with 943 nodes and 20319 edges
total_graph connected :  True  user graph info :  Graph with 2625 nodes and 165345 edges
user_graph connected :  False  user graph info :  Graph with 943 nodes and 25485 edges
total_graph connected :  True  user graph info :  Graph with 2625 nodes and 181516 edges
user_graph connected :  False  user graph info :  Graph with 943 nodes and 30174 edges
total_graph connected :  True  user graph info :  Graph with 2625 nodes and 197488 edges
user_graph connected :  False  u

In [53]:
from networkx.algorithms.community import asyn_lpa_communities,label_propagation_communities
from networkx.algorithms.community import coverage, performance

cluster = []
user_graph = G

In [54]:
comp = label_propagation_communities(user_graph)

In [55]:
result = list(comp)

In [56]:
len(result)

568

In [42]:
tmp = 1
for i in result:
    if len(i) == 1:
        tmp += 1
        
tmp

658

In [57]:
from networkx.algorithms.community import asyn_lpa_communities,label_propagation_communities
from networkx.algorithms.community import coverage, performance

cluster = []
total_graph = B

comp = label_propagation_communities(total_graph)

In [58]:
result = list(comp)

In [59]:
len(result)

1

In [ ]:
from networkx.algorithms.community import asyn_lpa_communities
from networkx.algorithms.community import coverage, performance

cluster = []
total_graph = B

In [12]:
comp = asyn_lpa_communities(total_graph)

In [13]:
result = list(comp)

In [14]:
print(len(result))

1


In [15]:
result

[{'i386',
  'u903',
  'i947',
  'u600',
  'u666',
  'i382',
  'i723',
  'i1260',
  'i399',
  'u353',
  'i1625',
  'u413',
  'i735',
  'u509',
  'i988',
  'i570',
  'i629',
  'u731',
  'i49',
  'u315',
  'i1286',
  'i1546',
  'u832',
  'i719',
  'i42',
  'u431',
  'i394',
  'u703',
  'i250',
  'i432',
  'i1196',
  'i1503',
  'i919',
  'i186',
  'i447',
  'u154',
  'i435',
  'i731',
  'u173',
  'u56',
  'u20',
  'u688',
  'i866',
  'i161',
  'i83',
  'i1284',
  'i1134',
  'u378',
  'u555',
  'i1390',
  'u583',
  'u178',
  'i761',
  'u483',
  'i1185',
  'u918',
  'u920',
  'i741',
  'i572',
  'u198',
  'i1298',
  'u882',
  'u705',
  'i1442',
  'i1478',
  'u243',
  'i199',
  'i559',
  'i355',
  'i1321',
  'i1223',
  'u720',
  'u497',
  'u116',
  'i67',
  'i1215',
  'i1668',
  'i1606',
  'i1613',
  'i1124',
  'u735',
  'u17',
  'u522',
  'u942',
  'u295',
  'u530',
  'i1421',
  'u101',
  'u788',
  'i1217',
  'i613',
  'u26',
  'i1147',
  'u157',
  'u441',
  'i1069',
  'u369',
  'u511',
  'i

In [16]:
##### 클러스터링 결과가 1개의 클러스터로 나옴,,, 무슨 의미즤...

In [17]:
from networkx.algorithms.community import asyn_lpa_communities
from networkx.algorithms.community import coverage, performance

cluster = []
user_graph =  G

In [18]:
comp = asyn_lpa_communities(user_graph)

In [19]:
result = list(comp)

In [20]:
len(result)

658

In [21]:
info(G)

C:\Users\kddin\AppData\Local\Temp\ipykernel_6688\3355025025.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  info(G)


'Graph with 943 nodes and 8702 edges'

In [22]:
len(list(nx.connected_components(G)))

658

In [23]:
### 그래프에서 연결된 얘들을 클러스터링 하는 것인가,,,